In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load Excel file
df = pd.read_excel(r"C:\Users\fahadee\Desktop\Inventory Managment System\Inventoty_Clean_Data.xlsx")
df['SaleDate'] = pd.to_datetime(df['SaleDate'])

# ------------------------------
# 1: Top-selling Categories (Bar Chart Improved)
category_sales = df.groupby('Category')['SellingPrice'].sum().sort_values(ascending=False).reset_index()
fig1 = px.bar(
    category_sales, x='Category', y='SellingPrice',
    color='SellingPrice', color_continuous_scale='Viridis',
    title='Total Sales by Category',
    text='SellingPrice'
)
fig1.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig1.update_layout(
    yaxis_title='Total Sales',
    xaxis_title='Category',
    margin=dict(l=60, r=40, t=120, b=120),
    xaxis_tickangle=-45,
    hovermode='x unified',
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

# ------------------------------
# 2: Stock Levels per Product (Horizontal Bar)
product_stock = df.groupby('ProductName')['StockQuantity'].sum().sort_values(ascending=True).reset_index()
fig2 = px.bar(
    product_stock, x='StockQuantity', y='ProductName',
    orientation='h', color='StockQuantity', color_continuous_scale='Magma',
    title='Stock Levels by Product',
    text='StockQuantity'
)
fig2.update_traces(texttemplate='%{text}', textposition='outside')
fig2.update_layout(
    yaxis_title='Product',
    xaxis_title='Stock Quantity',
    margin=dict(l=200, r=40, t=80, b=60),
    hovermode='y unified'
)

# ------------------------------
# 3: Daily Sales Trend (Enhanced, Bigger, More Detailed)
daily_sales = df.groupby('SaleDate')['SellingPrice'].sum().reset_index()
fig3 = go.Figure()

# Line with markers
fig3.add_trace(go.Scatter(
    x=daily_sales['SaleDate'],
    y=daily_sales['SellingPrice'],
    mode='lines+markers',
    line=dict(color='blue', width=2),
    marker=dict(color='orange', size=6),
    name='Daily Sales',
    hovertemplate='Date: %{x|%Y-%m-%d}<br>Sales: %{y:.2f}<extra></extra>'
))

# Peak day annotation
peak_day = daily_sales.loc[daily_sales['SellingPrice'].idxmax()]
fig3.add_trace(go.Scatter(
    x=[peak_day['SaleDate']],
    y=[peak_day['SellingPrice']],
    mode='markers+text',
    marker=dict(color='red', size=12, symbol='diamond'),
    text=[f"Peak: {peak_day['SellingPrice']:.2f}"],
    textposition='top center',
    textfont=dict(size=12, color='black'),
    name='Peak'
))

fig3.update_layout(
    title='Daily Sales Trend (Detailed)',
    xaxis_title='Date',
    yaxis_title='Selling Price',
    xaxis=dict(
        tickangle=-45,
        showgrid=True,
        gridcolor='lightgrey',
        rangeslider=dict(visible=True),
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type='date'
    ),
    yaxis=dict(showgrid=True, gridcolor='lightgrey'),
    margin=dict(l=60, r=40, t=120, b=120),
    hovermode='x unified',
    height=600
)

# ------------------------------
# 4: Supplier Discount vs Sales (Scatter, More Detailed)
supplier_stats = df.groupby('Supplier').agg({
    'Discount':'mean', 
    'SellingPrice':'sum', 
    'StockQuantity':'sum'
}).reset_index()

fig4 = px.scatter(
    supplier_stats, x='Discount', y='SellingPrice',
    size='StockQuantity', color='Supplier',
    hover_name='Supplier',
    title='Supplier Discount vs Total Sales',
    labels={
        'Discount':'Avg Discount (%)',
        'SellingPrice':'Total Sales',
        'StockQuantity':'Total Stock'
    },
    size_max=50
)
fig4.update_traces(
    marker=dict(line=dict(width=1, color='DarkSlateGrey')),
    hovertemplate='<b>%{hovertext}</b><br>Discount: %{x:.2f}%<br>Total Sales: %{y:.2f}<br>Total Stock: %{marker.size}<extra></extra>'
)
fig4.update_layout(
    yaxis_title='Total Sales',
    xaxis_title='Average Discount (%)',
    margin=dict(l=60, r=40, t=120, b=60),
    hovermode='closest',
    height=600
)

# ------------------------------
# 5: Treemap of Sales by Category → Region → Product (Simplified)
fig5 = px.treemap(
    df, path=['Category', 'Region', 'ProductName'], values='SellingPrice',
    color='SellingPrice', color_continuous_scale='Viridis',
    title='Sales Distribution by Category, Region, and Product'
)
fig5.update_layout(
    margin=dict(l=10, r=10, t=120, b=10),
    height=600
)

# ------------------------------
# Display all figures
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()
